# UNGP websocket bug report - Bokeh demo notebook

**Run this notebook with the `ais-tt-dev` kernel on UNGP** - this kernel has the required packages pre-installed.

Jupyter notebooks with interactive geographic map plots are disconnected from the UNGP Jupyter server when a modest number of points are plotted. This behaviour is seen with both Bokeh and Folium, and is a regression from previous functionality under Data Mechanics (prior to the Ocean Spark migration), where many tens of thousands of points could be plotted without disconnections.

If running the notebooks via the Jupyter Gateway, the following errors are typical in the terminal at the point where the visualisation cell is displayed:

```
[W 10:43:59.603 NotebookApp] Lost connection to Gateway: e7cef9f9-2155-4191-9ece-b904448644fe
[I 10:43:59.603 NotebookApp] Attempting to re-establish the connection to Gateway in 1.12 secs (1/5): e7cef9f9-2155-4191-9ece-b904448644fe
[I 10:44:00.724 NotebookApp] Connecting to wss://api.spotinst.io/ocean/spark/cluster/osc-b22517f3/notebook/api/kernels/e7cef9f9-2155-4191-9ece-b904448644fe/channels
```

A possible explanation for this behaviour is a change in proxy or load balancing configuration, whereby websockets (wss) requests are incorrectly rewritten to HTTP requests. The bug does not appear to be specific to any version of the python visualisation libraries used within the kernels/notebooks.

This demo notebook was developed to enable the bug to be reproduced on UNGP. It can also be run locally without issue. The number of positions to be rendered is set in the penultimate cell, and can be set to demonstrate either working or buggy behaviour (on UNGP). This notebook can display in excess of 1m data points when run locally, but is unable to display 25k when run on UNGP.

In [ ]:
!pip install xyzservices geopandas

In [ ]:
from io import StringIO
import sys
sys.path

sys.path.append("/home/app/.local/lib/python3.10/site-packages")

import geopandas as gpd
import numpy as np
import pandas as pd
import xyzservices.providers as xyz
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

## Define ports

In [2]:
# Subset of Atlantic port data
ports_csv = """locode,port_name,port_zone,lat,lon
BRFOR,Fortaleza,POINT (-38.58333333333334 -3.716666666666667),-3.71667,-38.58333
CVMIN,Mindelo,POINT (-25 16.88333333333333),16.88333,-25.0
ESLCG,La Coruna (A Coruna),POINT (-8.383333333333333 43.36666666666667),43.36667,-8.38333
FRBES,Brest,POINT (-4.483333333333333 48.4),48.4,-4.48333
GBFAL,Falmouth,"MULTIPOLYGON (((-5.0793065975538 50.1735580517526, -5.04454132092351 50.1538808246831, -5.05365808927061 50.1481791539993, -5.10130839849814 50.1565361640029, -5.11407187418409 50.1712940236859, -5.0793065975538 50.1735580517526)))",50.16143,-5.08006
MACAS,Casablanca,POINT (-7.6 33.58333333333334),33.58333,-7.6
PTOPO,Porto,POINT (-8.616666666666667 41.15),41.15,-8.61667
PTVEL,Velas,POINT (-28.21666666666667 38.68333333333333),38.68333,-28.21667
USPEF,Port Everglades,POINT (-80.13333333333334 26.1),26.1,-80.13333
USPNJ,Port Newark,POINT (-74.11666666666666 40.68333333333333),40.68333,-74.11667"""

# Read port data to a GeoPandas dataframe
ports_df = pd.read_csv(StringIO(ports_csv))
ports_df["port_zone"] = gpd.GeoSeries.from_wkt(
    data=ports_df["port_zone"], crs="epsg:4326"
)
ports_gdf = gpd.GeoDataFrame(ports_df, geometry="port_zone", crs="EPSG:4326")

# Add webmercator coordinates (used by Bokeh)
port_centroids_gs = ports_gdf["port_zone"].to_crs("epsg:3857").centroid
ports_gdf["x"] = port_centroids_gs.x
ports_gdf["y"] = port_centroids_gs.y

ports_gdf.info()
ports_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   locode     10 non-null     object  
 1   port_name  10 non-null     object  
 2   port_zone  10 non-null     geometry
 3   lat        10 non-null     float64 
 4   lon        10 non-null     float64 
 5   x          10 non-null     float64 
 6   y          10 non-null     float64 
dtypes: float64(4), geometry(1), object(2)
memory usage: 688.0+ bytes


,locode,port_name,port_zone,lat,lon,x,y
0,BRFOR,Fortaleza,POINT (-38.58333 -3.71667),-3.71667,-38.58333,-4.295077e+06,-4.140279e+05
1,CVMIN,Mindelo,POINT (-25.00000 16.88333),16.88333,-25.00000,-2.782987e+06,1.907249e+06
2,ESLCG,La Coruna (A Coruna),POINT (-8.38333 43.36667),43.36667,-8.38333,-9.332284e+05,5.367950e+06
3,FRBES,Brest,POINT (-4.48333 48.40000),48.40000,-4.48333,-4.990824e+05,6.173660e+06
4,GBFAL,Falmouth,"MULTIPOLYGON (((-5.07931 50.17356, -5.04454 50...",50.16143,-5.08006,-5.655097e+05,6.474280e+06


## Create some vessel journeys

In [3]:
def calculate_haversine_distance(
    lon1: float, lat1: float, lon2: float, lat2: float
) -> float:
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees).
    Vectorised with numpy.
    Variants of this function can be found on StackOverflow.
    """

    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Calculate area
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    area = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2

    # Calculate distance
    central_angle = 2 * np.arcsin(np.sqrt(area))
    radius = 6371  # Radius of earth in kilometers
    distance = central_angle * radius

    return distance


# Fake minimal data to represent some journey around the atlantic
journeys_csv = """imo,source,destination
1000000,GBFAL,USPEF
1000001,PTVEL,PTOPO
1000002,CVMIN,BRFOR
1000003,USPNJ,BRFOR
1000004,ESLCG,FRBES
"""

# Merge journeys with ports to get start and end locations
journeys_df = (
    pd.read_csv(StringIO(journeys_csv), dtype={"imo": str})
    .merge(
        ports_gdf.set_index("locode")[["lat", "lon"]],
        left_on="source",
        right_on="locode",
    )
    .merge(
        ports_gdf.set_index("locode")[["lat", "lon"]],
        left_on="destination",
        right_on="locode",
        suffixes=["_source", "_destination"],
    )
)

# Fake departure time
journeys_df["time_departure"] = pd.Timestamp("2023-01-01") + (
    journeys_df.index * pd.Timedelta("1D")
)

# Calculate journey distance
journeys_df["km"] = journeys_df.apply(
    lambda x: calculate_haversine_distance(
        *x[["lat_source", "lon_source", "lat_destination", "lon_destination"]]
    ),
    axis="columns",
)

journeys_df.info()
journeys_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   imo              5 non-null      object        
 1   source           5 non-null      object        
 2   destination      5 non-null      object        
 3   lat_source       5 non-null      float64       
 4   lon_source       5 non-null      float64       
 5   lat_destination  5 non-null      float64       
 6   lon_destination  5 non-null      float64       
 7   time_departure   5 non-null      datetime64[ns]
 8   km               5 non-null      float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 400.0+ bytes


,imo,source,destination,lat_source,lon_source,lat_destination,lon_destination,time_departure,km
0,1000000,GBFAL,USPEF,50.16143,-5.08006,26.10000,-80.13333,2023-01-01,8443.143183
1,1000001,PTVEL,PTOPO,38.68333,-28.21667,41.15000,-8.61667,2023-01-02,2194.700645
2,1000002,CVMIN,BRFOR,16.88333,-25.00000,-3.71667,-38.58333,2023-01-03,2453.996522
3,1000003,USPNJ,BRFOR,40.68333,-74.11667,-3.71667,-38.58333,2023-01-04,4578.540740
4,1000004,ESLCG,FRBES,43.36667,-8.38333,48.40000,-4.48333,2023-01-05,705.155737


## Synthesise positions for journeys

In [4]:
def generate_journey_df(row):
    """Calculate fake route AIS positions and timestamps for journey, return Pandas DataFrame.
    Speed and position report frequency are not typical of real world AIS data"""

    count = int(row["km"]) * 5
    lats = np.linspace(row["lat_source"], row["lat_destination"], count)
    lons = np.linspace(row["lon_source"], row["lon_destination"], count)
    ts = pd.date_range(
        start=row["time_departure"], periods=count, freq=pd.Timedelta("1m")
    )
    df = pd.DataFrame(
        data={
            "imo": row["imo"],
            "ts": ts,
            "lat": lats,
            "lon": lons,
        }
    )
    return df


# Concatenate route dataframes for each journey
positions_df = pd.concat(
    [generate_journey_df(row) for i, row in journeys_df.iterrows()]
)

# Convert to a GeoPandas dataframe
positions_gdf = gpd.GeoDataFrame(
    positions_df,
    geometry=gpd.points_from_xy(positions_df["lon"], positions_df["lat"]),
    crs="EPSG:4326",
)

# Add webmercator coordinates (used by Bokeh)
web_mercator_gs = positions_gdf.geometry.to_crs("epsg:3857")
positions_df["x"] = web_mercator_gs.x
positions_df["y"] = web_mercator_gs.y

positions_gdf.info()
positions_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 91865 entries, 0 to 3524
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   imo       91865 non-null  object        
 1   ts        91865 non-null  datetime64[ns]
 2   lat       91865 non-null  float64       
 3   lon       91865 non-null  float64       
 4   geometry  91865 non-null  geometry      
 5   x         91865 non-null  float64       
 6   y         91865 non-null  float64       
dtypes: datetime64[ns](1), float64(4), geometry(1), object(1)
memory usage: 5.6+ MB


,imo,ts,lat,lon,geometry,x,y
0,1000000,2023-01-01 00:00:00,50.16143,-5.080060,POINT (-5.08006 50.16143),-565509.692399,6.474280e+06
1,1000000,2023-01-01 00:01:00,50.16086,-5.081838,POINT (-5.08184 50.16086),-565707.609957,6.474181e+06
2,1000000,2023-01-01 00:02:00,50.16029,-5.083616,POINT (-5.08362 50.16029),-565905.527516,6.474082e+06
3,1000000,2023-01-01 00:03:00,50.15972,-5.085394,POINT (-5.08539 50.15972),-566103.445074,6.473983e+06
4,1000000,2023-01-01 00:04:00,50.15915,-5.087172,POINT (-5.08717 50.15915),-566301.362632,6.473884e+06


## Plot journeys

In [7]:
# Set plot parameters

# TODO change this

#position_count = 10000  # This renders
position_count = 25000 # This causes websocket disconnection
# position_count = -1 # All positions - this causes websocket disconnection

t = 1  # Tile selection - choose from selection list below

In [8]:
# Map tile provider
selection = [
    "CartoDB.Positron",
    "CartoDB.Voyager",
    "Esri.WorldImagery",
    "OpenStreetMap.Mapnik",
    "Stamen.Terrain",
    "Stamen.Toner",
    "Stamen.TonerLite",
]

providers = xyz.flatten()
tiles = providers[selection[t]]

# Create a Bokeh plot
p = figure(
    title=f"Fake vessel journeys ({selection[t]})",
    width=900,
    height=600,
    x_axis_type="mercator",
    y_axis_type="mercator",
)
p.grid.visible = False
p.add_tile(tiles)

# Render ports
ports_renderer = p.circle(
    name="ports",
    source=ColumnDataSource(ports_gdf.drop(columns=["port_zone"])),
    line_color="red",
    line_width=2,
)

# Iterate through journeys, plotting each as a separate bokeh Line
positions_renderers = []
for imo, group in (
    positions_gdf[:position_count].drop(columns="geometry").groupby("imo")
):
    positions_renderers.append(p.line(name=imo, source=ColumnDataSource(group)))

# Add HoverTools
hover_ports = HoverTool(
    name="ports",
    renderers=[ports_renderer],
    description="Ports",
    tooltips=[("Locode", "@locode"), ("Port Name", "@port_name")],
)

hover_positions = HoverTool(
    name="positions",
    renderers=positions_renderers,
    description="Positions",
    tooltips=[
        ("IMO", "@imo"),
        ("Timestamp", "@ts{%F %T}"),
        ("Lat", "@lat"),
        ("Lon", "@lon"),
    ],
    formatters={"@ts": "datetime"},
)

p.add_tools(*[hover_ports, hover_positions])

# Display the plot
show(p)